# Чистим датасет с фотографиями

- От баннеров. **`ocrmac` Работает только на macos!!**.
- От однотипных фотографий через поиск дубликатов.
- От ещё мусора, если определенные пиксели сильно выделяются. На фото распределение пикселей обычно не имеет какой-то моды.
- От фото с низким разрешением.

## Распознаем текст

In [ ]:
from tqdm.notebook import tqdm

OUTPUT_FOLDER = "../data/dataset_clean_info"

In [ ]:
# import glob
# import os
# from concurrent.futures import ThreadPoolExecutor
# from pathlib import Path

# import pandas as pd
# from ocrmac import ocrmac
# from tqdm.notebook import tqdm

# imgs_path = "../data/real_estate_images"
# OUTPUT_FOLDER = "../data/dataset_clean_info"

# os.makedirs(OUTPUT_FOLDER, exist_ok=True)


# def ocr_task(img_path: str) -> tuple[int, list[tuple[str, float]]]:
#     "Given image path, return (image_id, [(text, confidence)])"
#     img_id = int(Path(img_path).stem)
#     annotations = ocrmac.OCR(
#         img_path, language_preference=["ru-RU", "en-US"]
#     ).recognize()
#     return img_id, [i[0] for i in annotations]


# def process_images(images_path: str) -> pd.DataFrame:
#     """
#     Apply ocr_task to all images in a dataset_path, returns DF.
#     Uses maximum threads.
#     """
#     img_paths = glob.glob(f"{images_path}/*.jpg")

#     with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
#         results = list(tqdm(executor.map(ocr_task, img_paths), total=len(img_paths)))
#     return pd.DataFrame(results, columns=["image_id", "annotations"])


# df = process_images(imgs_path)
# df.to_csv(os.path.join(OUTPUT_FOLDER, "images_ocrmac.csv"), index=False)

In [ ]:
import glob
import ast
import random
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

In [ ]:
def filter_images_by_elements(df, fn_filter):
    df["annotations"] = df["annotations"].apply(ast.literal_eval)
    filtered_df = df[df["annotations"].apply(fn_filter)]
    return filtered_df


imgs_path = "../data/real_estate_images"
df = pd.read_csv("../data/dataset_clean_info/images_ocrmac.csv")
filtered_df = filter_images_by_elements(df, lambda x: len(x) > 2)
print(len(filtered_df))

In [ ]:
def display_images(image_ids, images_path="../data/real_estate_images"):
    all_images = glob.glob(f"{images_path}/*.jpg")

    selected_images = [img for img in all_images if int(Path(img).stem) in image_ids]

    if len(selected_images) > 9:
        selected_images = random.sample(selected_images, 9)

    print(selected_images)
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    axes = axes.flatten()

    for ax, img_path in zip(axes, selected_images):
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis("off")

    plt.tight_layout()
    plt.show()


image_ids = filtered_df["image_id"].tolist()
display_images(image_ids, imgs_path)

## Duplicates

In [ ]:
# import pandas as pd
# from pathlib import Path
# import imagehash
# from PIL import Image
# from tqdm.notebook import tqdm
# import glob


# def find_duplicates(images_directory, hash_size=8):
#     image_hashes = []

#     image_files = glob.glob(f"{images_directory}/*.jpg") + glob.glob(
#         f"{images_directory}/*.jpeg"
#     )

#     for image_path in tqdm(image_files):
#         image = Image.open(image_path)
#         image_id = int(Path(image_path).stem)
#         image_hash = imagehash.dhash(image, hash_size=hash_size)
#         image_hashes.append((image_id, str(image_hash)))
#         image.close()

#     df = pd.DataFrame(image_hashes, columns=["image_id", "hash"])
#     df["occurrences"] = df.groupby("hash")["hash"].transform("count")

#     return df


# images_directory = "../data/real_estate_images"
# OUTPUT_FOLDER = "../data/dataset_clean_info"

# Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

# df_duplicates = find_duplicates(images_directory)
# df_duplicates.to_csv(Path(OUTPUT_FOLDER) / "images_duplicates.csv", index=False)

In [ ]:
im_duplicates = pd.read_csv(Path(OUTPUT_FOLDER) / "images_duplicates.csv")
duplicates_ids = im_duplicates[im_duplicates.occurrences >= 2].image_id.to_list()
len(duplicates_ids)

In [ ]:
display_images(duplicates_ids)

## Color outliers

In [ ]:
# from multiprocess import Pool, cpu_count
# import glob
# from pathlib import Path
# import numpy as np
# import pandas as pd
# from PIL import Image
# from tqdm.notebook import tqdm


# def process_image(image_path, resize=(100, 100)):
#     """
#     Process a single image to determine if it's a color outlier.
#     """
#     image_id = int(Path(image_path).stem)
#     with Image.open(image_path) as image:
#         image = image.resize(resize)
#         pixels = np.array(image.convert("RGB"))

#     colors, counts = np.unique(pixels.reshape(-1, 3), axis=0, return_counts=True)
#     color_prob = counts / counts.sum()

#     # outlier = (color_prob > threshold).any()
#     return image_id, color_prob.max()


# def find_color_outliers(images_directory):
#     image_files = glob.glob(f"{images_directory}/*.jpg") + glob.glob(
#         f"{images_directory}/*.jpeg"
#     )

#     with Pool(processes=cpu_count()) as pool:
#         results = list(pool.starmap(process_image, [(path,) for path in image_files]))

#     df = pd.DataFrame(results, columns=["image_id", "top_color_prob"])
#     return df


# images_directory = "../data/real_estate_images"
# OUTPUT_FOLDER = "../data/dataset_clean_info"

# Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

# df_color_outliers = find_color_outliers(images_directory)
# df_color_outliers.to_csv(Path(OUTPUT_FOLDER) / "images_color_outliers.csv", index=False)

In [ ]:
df_color = pd.read_csv(Path(OUTPUT_FOLDER) / "images_color_outliers.csv")
color_ids = df_color[
    (df_color.top_color_prob > 0.1) & (df_color.top_color_prob <= 1)
].image_id.to_list()
print(len(color_ids))

In [ ]:
display_images(color_ids)

## smol

In [ ]:
# from multiprocess import Pool, cpu_count
# import glob
# from pathlib import Path
# import pandas as pd
# from PIL import Image
# from tqdm.notebook import tqdm


# def process_image_for_resolution(image_path):
#     """
#     Process a single image to determine its resolution.
#     """
#     image_id = int(Path(image_path).stem)
#     with Image.open(image_path) as image:
#         width, height = image.size
#     return image_id, width, height


# def find_image_resolutions(images_directory):
#     image_files = glob.glob(f"{images_directory}/*.jpg") + glob.glob(
#         f"{images_directory}/*.jpeg"
#     )

#     with Pool(processes=cpu_count()) as pool:
#         results = list(pool.map(process_image_for_resolution, image_files))

#     df = pd.DataFrame(results, columns=["image_id", "width", "height"])
#     return df


# images_directory = "../data/real_estate_images"
# OUTPUT_FOLDER = "../data/dataset_clean_info"

# Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

# df_image_resolutions = find_image_resolutions(images_directory)
# df_image_resolutions.to_csv(Path(OUTPUT_FOLDER) / "images_resolutions.csv", index=False)

In [ ]:
def select_low_resolution_images(csv_path, min_resolution):
    df = pd.read_csv(csv_path)
    filtered_df = df[(df["width"] < min_resolution) | (df["height"] < min_resolution)]
    low_res_image_ids = filtered_df["image_id"].tolist()
    return low_res_image_ids


csv_path = Path(OUTPUT_FOLDER) / "images_resolutions.csv"
low_res_image_ids = select_low_resolution_images(csv_path, min_resolution=350)
print(len(low_res_image_ids))

In [ ]:
display_images(low_res_image_ids)

## copy clean images

In [ ]:
import ast
import shutil
from pathlib import Path

import pandas as pd

# Define paths
imgs_path = Path("../data/real_estate_images")
OUTPUT_FOLDER = Path("../data/dataset_clean_info")
clean_imgs_path = imgs_path.parent / "real_estate_images_clean"
if clean_imgs_path.exists():
    shutil.rmtree(clean_imgs_path)
clean_imgs_path.mkdir(parents=True, exist_ok=True)


# Detector 1: OCR-based filtering
def filter_images_by_elements(df, fn_filter):
    df["annotations"] = df["annotations"].apply(ast.literal_eval)
    filtered_df = df[df["annotations"].apply(fn_filter)]
    return filtered_df


df_ocr = pd.read_csv(OUTPUT_FOLDER / "images_ocrmac.csv")
filtered_ocr_ids = filter_images_by_elements(df_ocr, lambda x: len(x) > 2)[
    "image_id"
].tolist()
print("OCR Detector:", len(filtered_ocr_ids))

# Detector 2: Duplicate detection
df_duplicates = pd.read_csv(OUTPUT_FOLDER / "images_duplicates.csv")
duplicates_ids = df_duplicates[df_duplicates.occurrences >= 2]["image_id"].tolist()
print("Duplicates Detector:", len(duplicates_ids))

# Detector 3: Color outlier detection
df_color = pd.read_csv(OUTPUT_FOLDER / "images_color_outliers.csv")
color_ids = df_color[(df_color.top_color_prob > 0.1) & (df_color.top_color_prob <= 1)][
    "image_id"
].tolist()
print("Color Outliers Detector:", len(color_ids))


# Detector 4: Resolution detection
def select_low_resolution_images(csv_path, min_resolution):
    df = pd.read_csv(csv_path)
    filtered_df = df[(df["width"] < min_resolution) | (df["height"] < min_resolution)]
    return filtered_df["image_id"].tolist()


low_res_image_ids = select_low_resolution_images(
    OUTPUT_FOLDER / "images_resolutions.csv", min_resolution=350
)
print("Low Resolution Detector:", len(low_res_image_ids))

# Union of all detected IDs
all_detected_ids = set(
    filtered_ocr_ids + duplicates_ids + color_ids + low_res_image_ids
)
image_files = glob.glob(str(imgs_path / "*.jpg")) + glob.glob(str(imgs_path / "*.jpeg"))
print(
    "Total unique IDs detected by all detectors:",
    len(all_detected_ids),
    f"({len(all_detected_ids) / len(image_files) * 100:.2f}%)",
)
print(f"{ len(image_files) - len(all_detected_ids)} left")

with open(Path(clean_imgs_path) / ".gitkeep", "w") as _:
    ...

# Copy images excluding detected IDs
for image_path in tqdm(image_files):
    image_id = int(Path(image_path).stem)
    if image_id not in all_detected_ids:
        shutil.copy(image_path, clean_imgs_path)

## shuffle dataset

In [ ]:
import random
import uuid
from pathlib import Path

from tqdm.notebook import tqdm


def shuffle_and_rename_images(folder_path):
    """
    Shuffle and rename all images in the specified folder to a sequence of numbers.
    This version uses a temporary UUID to avoid overwriting files during the process.

    Parameters:
        folder_path (str or Path): The path to the folder containing the images.
    """
    folder_path = Path(folder_path)

    image_files = list(folder_path.glob("*.jpg")) + list(folder_path.glob("*.jpeg"))

    random.shuffle(image_files)

    # First, rename all files to a temporary unique UUID to avoid any conflict
    temp_files = []
    for file_path in tqdm(image_files):
        temp_file_path = folder_path / f"{uuid.uuid4()}.tmp"
        file_path.rename(temp_file_path)
        temp_files.append(temp_file_path)

    # Now, rename all temporary files to the desired sequence
    for idx, temp_file_path in enumerate(tqdm(temp_files), start=1):
        new_file_path = folder_path / f"{idx}.jpg"
        temp_file_path.rename(new_file_path)

    print(f"Renamed {len(image_files)} images in {folder_path}")


clean_images_folder = "../data/real_estate_images_clean"
shuffle_and_rename_images(clean_images_folder)